In [ ]:
import pandas as pd

In [ ]:
%run ../modules/utils.ipynb
%run ../modules/cds.ipynb
%run ../modules/preprocessing.ipynb

%run ../modules/paper_figures.ipynb

In [ ]:
import time

In [ ]:
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/23_t_Cys.xlsx'
df_wild = load_data(path)
path = '/Users/xyuan/Documents/ChenLab/10Jun2021/degraded/24_t_Cys_A.xlsx'
df_alkb = load_data(path)
df_wild.shape, df_alkb.shape

In [ ]:
# path = '~/Studio/seq_bank/data/cys/ala_AGC1.xlsx'
# path = '~/Studio/seq_bank/data/cys/ala_AGC1_5p.xlsx'
# path = '~/Studio/seq_bank/data/cys/ala_AGC1_3p.xlsx'
# path = '~/Studio/seq_bank/data/cys/ala_AGC2_5p.xlsx'
path = '~/Studio/seq_bank/data/cys/ala_AGC2_3p.xlsx'
# path = '~/Studio/seq_bank/data/cys/ala_CGC1_3p.xlsx'
# path = '~/Studio/seq_bank/data/cys/ala_TGC1_3p.xlsx'

# path = '~/Studio/seq_bank/data/cys/cys_GCA1.xlsx'
# path = '~/Studio/seq_bank/data/cys/cys_GCA1_5p.xlsx'
# path = '~/Studio/seq_bank/data/cys/cys_GCA1_3p.xlsx'
df_ngs1 = load_data(path)

ala_agc1_5p = 'GGGGGUGUAGCUCAGUGGUAGAGCGCGUGCUUAGCAUGCACGAGGCCCUGGGUUCGAUCCCCAGCACCUCCACCA'
ala_agc1_3p = ala_agc1_5p[::-1]
ala_agc2_5p = 'GGGGAUGUAGCUCAGUGGUAGAGCGCAUGCUUAGCAUGCAUGAGGUCCUGGGUUCGAUCCCCAGCAUCUCCACCA'
ala_agc2_3p = ala_agc2_5p[::-1]
ala_cgc1_5p = 'GGGGAUGUAGCUCAGUGGUAGAGCGCAUGCUUCGCAUGUAUGAGGCCCCGGGUUCGAUCCCCGGCAUCUCCACCA'
ala_cgc1_3p = ala_cgc1_5p[::-1]
ala_tgc1_5p = 'GGGGGUGUAGCUCAGUGGUAGAGCGCAUGCUUUGCAUGCAUGAGGCCCCGGGUUCGAUCCCCGGCACCUCCACCA'
ala_tgc1_3p = ala_tgc1_5p[::-1]

cys_gca1_5p = 'GGGGGUAUAGCUCAGUGGUAGAGCAUUUGACUGCAGAUCAAGAGGUCCCCGGUUCAAAUCCGGGUGCCCCCUCCA'
cys_gca1_3p = cys_gca1_5p[::-1]

ngs_seq = ala_agc2_3p

In [ ]:
df_sample = df_ngs1
plotly_zone(df_sample, y='RT')
df_sample.shape
df_ngs1.head()
# bcr = base_calling_random(df_ngs1)
# plotly_basecalling(*bcr)
# print(bcr[1])

df_ngs = df_ngs1.copy()
df_ngs['Base'] = list(ngs_seq)
df_ngs['Idx'] = 75-np.array(df_ngs.index)
df_ngs

In [ ]:
df_ngs[['Mass', 'Base', 'Idx']].to_excel('~/Downloads/tmp.xlsx')

In [ ]:
df_wild_sample = local_top(df_wild, top=20)
df_alkb_sample = local_top(df_alkb, top=20)
# df_wild_sample = df_wild.copy()
# df_alkb_sample = df_wild.copy()
df_wild_sample.shape, df_alkb_sample.shape

In [ ]:
import multiprocessing
from collections import namedtuple
ShiftHit = namedtuple('ShiftHit', 'shift hit')

def func(df_s1, df_s2, shift):
    dfm = match_dfs(df_s1, df_s2, shift=shift)
    sh = ShiftHit(shift=shift, hit=dfm.shape[0])
    return sh

PROCESSES = 8
params = [(df_ngs, df_wild_sample, shift) for shift in np.arange(0, 160, 0.1)]
# params = [(df_wild_sample, df_wild_sample, shift) for shift in np.arange(-300, 300, 1)]
with multiprocessing.Pool(PROCESSES) as pool:
    shift_hits = pool.starmap(func, params)

df_shift_hits = pd.DataFrame(shift_hits)

In [ ]:
# shift_hits

In [ ]:
fig = px.scatter(df_shift_hits[df_shift_hits.hit<df_shift_hits.shape[0]], x='shift', y='hit', labels={'shift': 'Mass Shift', 'hit': 'Hit Count'})
fig.show()

In [ ]:
# shifts = [-60, 0, 13, 27, 41.1, 42]
# df_sh_top = df_shift_hits[df_shift_hits['shift']>=1].sort_values('hit', ascending=False).iloc[:10]
df_sh_top = df_shift_hits[df_shift_hits.hit>4]
shifts = df_sh_top['shift']
df_sh_top.shape

In [ ]:
shifts = [0, 14, 28, 42, 56, 70, 84, 98]

In [ ]:
# find out data points which can generate base calling
def func(df_s1, df_s2, shift):
    _, df = peer_dfs(df_s1, df_s2, shift=shift)
    
#     grps = basecalling_groups(df)
#     if grps[0][0].shape[0] <= 5:
#         return pd.DataFrame()
    bcr = base_calling_random(df, base_only=True)
#     dfms.append(bcr[0])
    
    df = bcr[0].copy()
    df['Shift'] = shift
    return df

PROCESSES = 8
params = [(df_ngs, df_wild_sample, shift) for shift in shifts]
with multiprocessing.Pool(PROCESSES) as pool:
    df_basecallings = pool.starmap(func, params)

df_hits = pd.concat(df_basecallings).drop_duplicates()
df_hits.shape

In [ ]:
def valid_shifted_pairs(df_baseline, pairs):
    return [pair for pair in pairs if valid_shifted_pair(df_baseline, pair)]

def valid_shifted_pair(df_baseline, pair):
    """given a baseline sequence, and a basecalling pair, check if this pair 
    is somehow shifted from baseline sequence.
    Param df_baseline: a baseline sequence
    Param pair: an tuple like (mass1, mass2, nucleobase)
    
    Return: bool value. if pair valid or not
    """
    mass = pair[0]
    max_shift = 150
    baseline_base = df_baseline[(df_baseline.Mass>mass-max_shift)&(df_baseline.Mass<mass+max_shift)] 
#     print(baseline_base[:1].Base.iloc[0], type(baseline_base[:1].Base.iloc[0]))
    base = baseline_base[:1].Base.iloc[0] if not baseline_base.empty else ''
    observed_base = pair[2]
#     print(base, observed_base)
    return base in observed_base

In [ ]:
df_hits.to_excel('~/Downloads/hits_ala_tgc1.xlsx')

In [ ]:
plotly_zones(df_wild_sample, df_hits)
# df_hits.head()
# plotly_zone(df_ngs1)
plotly_zone(df_hits, y='Shift')
df_hits.shape

In [ ]:
# plotly_zone(df_hits)
# df_hits.shape

l, h = 100, 25000
bcr = base_calling_random(df_hits[(df_hits.Mass>l)&(df_hits.Mass<h)])
plot_basecalling(*bcr, figsize=(24, 18))
# pairs = valid_shifted_pairs(df_ngs, bcr[1])
# plotly_basecalling(bcr[0], pairs)

In [ ]:
grps = basecalling_groups(df_hits)

In [ ]:
plotly_basecalling(*grps[0])

# for i in range(len(dfms)):
#     dft = dfms[i]
#     bcr = base_calling_random(dft)
#     plotly_basecalling(dft, bcr[1])
#     time.sleep(1)

In [ ]:
shift = 100

dfm_l, _ = peer_dfs(df_wild_sample, df_wild_sample, shift=shift)
plotly_zone(dfm_l)

In [ ]:
dfm_l.shape